In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import plot_tree
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import LabelEncoder
# Cargar archivo csv
df = pd.read_csv("train.csv", encoding="latin1")


In [8]:
# Seleccionar solo las variables numéricas relevantes
numericas = df.select_dtypes(include='number')
numericas = numericas.drop(["Fireplaces", "GarageYrBlt", "Id", "MSSubClass", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "MoSold", "YrSold"], axis=1)

In [9]:
# Crear variable respuesta que clasifica las casas en Económicas, Intermedias o Caras
terciles = np.percentile(numericas["SalePrice"], [33.33, 66.67])
limite_economicas = terciles[0]
limite_caras = terciles[1]
numericas["Clase"] = pd.cut(numericas["SalePrice"], bins=[0, limite_economicas, limite_caras, float("inf")], labels=["Económicas", "Intermedias", "Caras"])

In [10]:
# Dividir en conjunto de entrenamiento y prueba
X = numericas.drop(["SalePrice", "Clase"], axis=1)
Y = numericas["Clase"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7)

# Imputar valores faltantes utilizando la media de cada columna
imputer = SimpleImputer(strategy="mean")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Crear modelo de árbol de clasificación
arbol = DecisionTreeClassifier(random_state=0)

# Entrenar modelo
arbol.fit(X_train, y_train)

# Predecir valores para conjunto de prueba
y_pred = arbol.predict(X_test)

# Evaluar precisión del modelo
precision = accuracy_score(y_test, y_pred)

In [11]:
# Predecir etiquetas para conjunto de prueba
y_pred = arbol.predict(X_test)

# Calcular precisión del modelo
precision = accuracy_score(y_test, y_pred)

### Inciso 9
Haga un análisis de la eficiencia del algoritmo usando una matriz de confusión para el árbol
de clasificación. Tenga en cuenta la efectividad, donde el algoritmo se equivocó más, donde
se equivocó menos y la importancia que tienen los errores.


In [12]:
# Creamos la matriz de confusion
cm = confusion_matrix(y_test, y_pred)

In [13]:
# Imprimimos la matriz de confusión
print("Matriz de confusión:")
print(cm)

Matriz de confusión:
[[118   4  27]
 [  1 113  33]
 [ 14  25 103]]


In [14]:
# Calculamos algunas métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [15]:
# Imprimimos las métricas de evaluación
print("Exactitud: {:.2f}".format(accuracy))
print("Precisión: {:.2f}".format(precision))
print("Sensibilidad: {:.2f}".format(recall))
print("Puntuación F1: {:.2f}".format(f1))

Exactitud: 0.76
Precisión: 0.77
Sensibilidad: 0.76
Puntuación F1: 0.77


### Inciso 10
Entrene un modelo usando validación cruzada, prediga con él. ¿le fue mejor que al modelo anterior?

In [16]:
# Dividir en conjunto de entrenamiento y prueba
X = numericas.drop(["SalePrice", "Clase"], axis=1).fillna(0)
y = numericas["Clase"]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, train_size=0.7)

In [18]:
# Creamos un clasificador de árbol de decisión
clf = DecisionTreeClassifier()

In [19]:
# Realizamos la validación cruzada
scores = cross_val_score(clf, X, y, cv=5, error_score='raise')

In [20]:
# Imprimimos la puntuación de precisión para cada pliegue
print("Precisión de cada pliegue: {}".format(scores))
print("Precisión promedio: {:.2f}".format(np.mean(scores)))

Precisión de cada pliegue: [0.7739726  0.74315068 0.80821918 0.74315068 0.75342466]
Precisión promedio: 0.76


In [21]:
# Entrenamos el clasificador con los datos de entrenamiento
clf.fit(X_train, y_train)

# Hacemos predicciones sobre los datos de prueba
y_pred = clf.predict(X_test)

In [22]:
# Imprimimos la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

Matriz de confusión:
[[142   4  19]
 [  1 116  25]
 [ 21  24  86]]


In [23]:
# Calculamos algunas métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [24]:
# Imprimimos las métricas de evaluación
print("Analisis;")
print("------------------------------------")
print("Exactitud: {:.2f}".format(accuracy))
print("Precisión: {:.2f}".format(precision))
print("Sensibilidad: {:.2f}".format(recall))
print("Puntuación F1: {:.2f}".format(f1))

Analisis;
------------------------------------
Exactitud: 0.79
Precisión: 0.79
Sensibilidad: 0.79
Puntuación F1: 0.79


### Inciso 11
Haga al menos, 3 modelos más cambiando la profundidad del árbol. ¿Cuál funcionó mejor?

In [25]:
depths = [6, 8, 10]
'''
Si cambia el valor de max_depth, es posible que se deba ajustar el 
número de pliegues (cv) en la validación cruzada. La elección del 
valor de cv puede depender de varios factores, incluyendo la cantidad 
de datos disponibles, la complejidad del modelo y la cantidad de 
recursos de cómputo disponibles.
En general, se recomienda utilizar entre 5 y 10 pliegues en la validación cruzada
para la mayoría de los conjuntos de datos.

'''
for i in depths:
    print("Profundidad: ", i)
    # Creamos un clasificador de árbol de decisión
    clf = DecisionTreeClassifier(max_depth=i)
    # Realizamos la validación cruzada
    scores = cross_val_score(clf, X, y, cv=i, error_score='raise')
    # Imprimimos la puntuación de precisión para cada pliegue
    print("Precisión de cada pliegue: {}".format(scores))
    print("Precisión promedio: {:.2f}".format(np.mean(scores)))
    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)
    # Hacemos predicciones sobre los datos de prueba
    y_pred = clf.predict(X_test)
    # Imprimimos la matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    # Calculamos algunas métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Imprimimos las métricas de evaluación
    print("Exactitud: {:.2f}".format(accuracy))
    print("Precisión: {:.2f}".format(precision))
    print("Sensibilidad: {:.2f}".format(recall))
    print("Puntuación F1: {:.2f}".format(f1))
    print("--------------------------------------")

Profundidad:  6
Precisión de cada pliegue: [0.77459016 0.77868852 0.78600823 0.77366255 0.75308642 0.75308642]
Precisión promedio: 0.77
Exactitud: 0.77
Precisión: 0.79
Sensibilidad: 0.77
Puntuación F1: 0.78
--------------------------------------
Profundidad:  8
Precisión de cada pliegue: [0.7704918  0.84153005 0.75956284 0.78142077 0.8021978  0.73626374
 0.79120879 0.78021978]
Precisión promedio: 0.78
Exactitud: 0.79
Precisión: 0.79
Sensibilidad: 0.79
Puntuación F1: 0.79
--------------------------------------
Profundidad:  10
Precisión de cada pliegue: [0.74657534 0.81506849 0.76027397 0.7739726  0.78767123 0.81506849
 0.7260274  0.80821918 0.69178082 0.7260274 ]
Precisión promedio: 0.77
Exactitud: 0.78
Precisión: 0.78
Sensibilidad: 0.78
Puntuación F1: 0.78
--------------------------------------


In [26]:
print("Analisis;")
print("------------------------------------")
print("La profundidad que mejor se adaptó fue de grado 8. \
    Se observa que la precision promedio del modelo cruzado fue \
    del 0.79, ligeramente mejor que los otros modelos.")

Analisis;
------------------------------------
La profundidad que mejor se adaptó fue de grado 8.     Se observa que la precision promedio del modelo cruzado fue     del 0.79, ligeramente mejor que los otros modelos.


### Inciso 12
Repita los análisis usando random forest como algoritmo de predicción, explique sus resultados comparando ambos algoritmos.

In [27]:
# Dividir en conjunto de entrenamiento y prueba
X = numericas.drop(["SalePrice", "Clase"], axis=1).fillna(0)
y = numericas["Clase"]
# Convertir las etiquetas de texto en valores numéricos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [28]:
X

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,GarageCars,GarageArea,PoolArea,MiscVal
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,...,0,2,1,3,1,8,2,548,0,0
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,...,1,2,0,3,1,6,2,460,0,0
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,...,0,2,1,3,1,6,2,608,0,0
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,...,0,1,0,3,1,7,3,642,0,0
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,...,0,2,1,4,1,9,3,836,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,6,5,1999,2000,0.0,0,0,953,...,0,2,1,3,1,7,2,460,0,0
1456,85.0,13175,6,6,1978,1988,119.0,790,163,589,...,0,2,0,3,1,7,2,500,0,0
1457,66.0,9042,7,9,1941,2006,0.0,275,0,877,...,0,2,0,4,1,9,1,252,0,2500
1458,68.0,9717,5,6,1950,1996,0.0,49,1029,0,...,0,1,0,2,1,5,1,240,0,0


In [29]:
y

0             Caras
1       Intermedias
2             Caras
3       Intermedias
4             Caras
           ...     
1455    Intermedias
1456          Caras
1457          Caras
1458    Intermedias
1459    Intermedias
Name: Clase, Length: 1460, dtype: category
Categories (3, object): ['Económicas' < 'Intermedias' < 'Caras']

In [30]:
y_encoded

array([0, 2, 0, ..., 0, 2, 2])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, train_size=0.7)

In [32]:
# Creamos una instancia del modelo de Random Forest, y usamos el conjunto de entrenamiento para entrenarlo.
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [33]:
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [34]:
# Usar el modelo entrenado para realizar las predicciones en el conjunto de prueba.
predictions = model.predict(X_test)

In [35]:
# Evaluar el rendimiento del modelo usando las métricas de evaluación 
# adecuadas para el tipo de problema que se esta resolviendo.
r_squared_score = r2(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
print(f"R2 score: {r_squared_score}")
print(f"MSE: {mse}")

R2 score: 0.5823886921261826
MSE: 0.285062100456621


In [36]:
print("Random forest es una tecnica de aprendizaje muy usada. \
    Su flexibilidad y uso permiten que pueda manejar tanto problemas \
    de clasificacion, como de regresion. \
    \n Al compararlo con otros modelos, se puede observar que no \
    rindio tan bien comparado con el modelo de validacion cruzada ni \
    como el arbol. \
    \n Se puede concluir que el mejor modelo implementado fue el de \
    validacion cruzada con una profundidad de 8 y 8 pliegues.")

Random forest es una tecnica de aprendizaje muy usada.     Su flexibilidad y uso permiten que pueda manejar tanto problemas     de clasificacion, como de regresion.     
 Al compararlo con otros modelos, se puede observar que no     rindio tan bien comparado con el modelo de validacion cruzada ni     como el arbol.     
 Se puede concluir que el mejor modelo implementado fue el de     validacion cruzada con una profundidad de 8 y 8 pliegues.
